In [1]:
import os
import cv2
import random
import shutil
import xml.dom.minidom
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [3]:
def scan_files(directory, prefix=None, postfix=None):
    files_list = []
    for root, sub_dirs, files in os.walk(directory):
        for special_file in files:
            if postfix:
                if special_file.endswith(postfix):
                    files_list.append(os.path.join(root, special_file))
            elif prefix:
                if special_file.startswith(prefix):
                    files_list.append(os.path.join(root, special_file))
            else:
                files_list.append(os.path.join(root, special_file))
    return files_list

### move jpgs with xmls/txts out to separate directory

In [2]:
def copy_jpg_and_labels(from_path, to_path):
    image_names = scan_files(from_path, postfix=".bmp")
    for image_name in image_names:
        xml_name = os.path.splitext(image_name)[0] + ".xml"
        txt_name = os.path.splitext(image_name)[0] + ".txt"
        if os.path.isfile(xml_name):
            shutil.move(image_name, to_path)
            shutil.move(xml_name, to_path)
        elif os.path.isfile(txt_name):
            shutil.move(image_name, to_path)
            shutil.move(txt_name, to_path)


orig_path = "../../yolo_sc_20181128/SC_orig"
tmp_path = "../../yolo_sc_20181128/SC_tmp"

copy_jpg_and_labels(orig_path, tmp_path)

### find proper training input image size (608x608)

In [ ]:
# collect all sizes of images
tmp_path = "../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done"
image_names = [os.path.join(tmp_path, f) for f in os.listdir(tmp_path) if f.endswith(".jpg")]
print("# of jpgs", len(image_names))
image_sizes = []
for image_name in image_names:
    with Image.open(image_name) as image:
        image_sizes.append(image.size)

In [ ]:
# kmeans clustering
X = np.array(image_sizes)
kmeans = KMeans(n_clusters=9, random_state=2018).fit(X)
print(X.shape, kmeans)

In [ ]:
centers = []
for center in kmeans.cluster_centers_:
    print(center)
    centers.append(center)

In [ ]:
total = len(image_sizes)
large = len([image_size for image_size in image_sizes if image_size[0] > 608 or image_size[1] > 608])
print(total, large)

### image padding and resizing

In [4]:
def read_labels_from_xml(xml_name):
    DOMTree = xml.dom.minidom.parse(xml_name)
    collection = DOMTree.documentElement
    filename = collection.getElementsByTagName("filename")
    objects = collection.getElementsByTagName("object")

    w = collection.getElementsByTagName("width")[0]
    w_val = int(w.firstChild.nodeValue)
    h = collection.getElementsByTagName("height")[0]
    h_val = int(h.firstChild.nodeValue)
    w.firstChild.replaceWholeText(str(h_val))
    h.firstChild.replaceWholeText(str(w_val))

    labels = {'w':w_val, 'h':h_val, 'boxes':[]}
    
    for object in objects:
        xmin = object.getElementsByTagName("xmin")[0]
        xmin_val = int(xmin.firstChild.nodeValue)
        xmax = object.getElementsByTagName("xmax")[0]
        xmax_val = int(xmax.firstChild.nodeValue)
        ymin = object.getElementsByTagName("ymin")[0]
        ymin_val = int(ymin.firstChild.nodeValue)
        ymax = object.getElementsByTagName("ymax")[0]
        ymax_val = int(ymax.firstChild.nodeValue)
        labels["boxes"].append([xmin_val, ymin_val, xmax_val, ymax_val])
        
    return labels


def read_labels_from_txt(txt_name):
    jpg_name = os.path.splitext(txt_name)[0] + ".bmp"
    with Image.open(jpg_name) as img:
        w, h = img.size
        
    labels = {'w':w, 'h':h, 'boxes':[]}
    with open(txt_name, 'r') as f:
        for line in f.readlines():
            tokens = line.strip().split()
            xc, yc = w*float(tokens[1]), h*float(tokens[2])
            w_, h_ = w*float(tokens[3]), h*float(tokens[4])
            xmin, ymin = int(xc - w_/2), int(yc - h_/2)
            xmax, ymax = int(xc + w_/2), int(yc + h_/2)
            labels["boxes"].append([xmin, ymin, xmax, ymax])
            
    return labels


def get_labels_from_cell(img_name):
    with Image.open(img_name) as img:
        w, h = img.size
        
    labels = {'w':w, 'h':h, 'boxes':[[0, 0, w, h],]}
    return labels


def resize_img_with_padding(img_name, size, save_path):
    xml_name = os.path.splitext(img_name)[0] + ".xml"
    txt_name = os.path.splitext(img_name)[0] + ".txt"
    if os.path.isfile(xml_name):
        labels = read_labels_from_xml(xml_name)
    elif os.path.isfile(txt_name):
        labels = read_labels_from_txt(txt_name)
    else:
        labels = get_labels_from_cell(img_name)
    
    img_name_new = os.path.join(save_path, os.path.basename(img_name))
    txt_name_new = os.path.splitext(img_name_new)[0] + ".txt"
    
    img = Image.open(img_name)
    
    # pad and crop image
    x = -((size - img.size[0]) // 2)
    y = -((size - img.size[1]) // 2)
    img_croped = img.crop(
        (
            x,
            y,
            size + x,
            size + y
        )
    )
    img_croped.save(img_name_new)

    # change label coordinates
    labels_yolo = []
    for box in labels["boxes"]:
        xmin = box[0] - x
        ymin = box[1] - y
        xmax = box[2] - x
        ymax = box[3] - y
        xcenter = (xmin+xmax)/2.0/size
        ycenter = (ymin+ymax)/2.0/size
        w = (xmax-xmin)/size
        h = (ymax-ymin)/size
        labels_yolo.append(['0', str(xcenter), str(ycenter), str(w), str(h)])
        
#         # cut cells
#         cell_name = os.path.splitext(img_name_new)[0] + '_' + str(xmin) + '_' + str(ymin) + ".jpg"
#         img_croped.crop((xmin, ymin, xmax, ymax)).save(cell_name)
        
    # write lables to txt
    with open(txt_name_new, 'w') as f:
        for label in labels_yolo:
            f.write(' '.join(label) + '\n')
            
#     print("processed ", img_name)

In [5]:
tmp_path = "/home/nvme/PH_train/PH"
yolo_path = "/home/nvme/PH_train/PH-train"
size = 608

image_names = scan_files(tmp_path, postfix=".bmp")
print("# files", len(image_names))
os.makedirs(yolo_path, exist_ok=True)

for image_name in image_names:
    resize_img_with_padding(image_name, size, yolo_path)

# files 4793


### split data into train/valid

In [ ]:
import random
import shutil


def split_train_valid(data_path):
    img_names = [os.path.join(data_path, f) for f in os.listdir(data_path) if f.endswith(".bmp")]

    random.shuffle(img_names)
    random.shuffle(img_names)

    split = 0.8

    train_names = img_names[:int(len(img_names)*split)]
    valid_names = img_names[int(len(img_names)*split):]

    train_path = os.path.join(data_path, "train")
    os.makedirs(train_path, exist_ok=True)
    for img_name in train_names:
        txt_name = os.path.splitext(img_name)[0] + ".txt"
        shutil.move(img_name, train_path)
        shutil.move(txt_name, train_path)

    valid_path = os.path.join(data_path, "valid")
    os.makedirs(valid_path, exist_ok=True)
    for img_name in valid_names:
        txt_name = os.path.splitext(img_name)[0] + ".txt"
        shutil.move(img_name, valid_path)
        shutil.move(txt_name, valid_path)
        

yolo_path = "../../yolo_hsil_s_20181128/hsil_s_yolo"
split_train_valid(yolo_path)

### kmeans clustering

In [6]:
def scan_files(directory, prefix=None, postfix=None):
    files_list = []
    for root, sub_dirs, files in os.walk(directory):
        for special_file in files:
            if postfix:
                if special_file.endswith(postfix):
                    files_list.append(os.path.join(root, special_file))
            elif prefix:
                if special_file.startswith(prefix):
                    files_list.append(os.path.join(root, special_file))
            else:
                files_list.append(os.path.join(root, special_file))
    return files_list

def collect_sizes_from_yolotxt(txt_name, size):
    sizes = []
    with open(txt_name, 'r') as f:
        for line in f.readlines():
            tokens = line.strip().split()
#             print(tokens)
            w = int(float(tokens[3])*size)
            h = int(float(tokens[4])*size)
            sizes.append([w, h])
    return sizes

def collect_sizes_from_yolotxt_all(txt_names, size):
    sizes = []
    for txt_name in txt_names:
        sizes += collect_sizes_from_yolotxt(txt_name, size)
    return sizes

In [7]:
yolo_path = "/home/nvme/PH_train/PH-train"
size = 608

sizes = collect_sizes_from_yolotxt_all(scan_files(yolo_path, postfix=".txt"), size)

# kmeans clustering
X = np.array(sizes)
kmeans = KMeans(n_clusters=9, random_state=2018).fit(X)
print(kmeans)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=9, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=2018, tol=0.0001, verbose=0)


In [8]:
centers = []
for center in kmeans.cluster_centers_:
    print(center)
    centers.append(center)

[127.64772727 161.71732955]
[175.01140684 173.99746515]
[189.97546012 225.25357873]
[115.90107527 114.4516129 ]
[245.31620553 247.50197628]
[198.75       138.72477064]
[157.09577465 135.03521127]
[148.31234867 203.01210654]
[222.85018727 184.62921348]


In [9]:
tosort = {int(center[0]*center[1]):center for center in centers}
print(tosort)

{13265: array([115.90107527, 114.4516129 ]), 20642: array([127.64772727, 161.71732955]), 30451: array([175.01140684, 173.99746515]), 30109: array([148.31234867, 203.01210654]), 42792: array([189.97546012, 225.25357873]), 41144: array([222.85018727, 184.62921348]), 60716: array([245.31620553, 247.50197628]), 27571: array([198.75      , 138.72477064]), 21213: array([157.09577465, 135.03521127])}


In [10]:
hassorted = sorted(tosort.items())
print(hassorted)
print(",  ".join(["{},{}".format(int(value[1][0]),int(value[1][1])) for value in hassorted]))

[(13265, array([115.90107527, 114.4516129 ])), (20642, array([127.64772727, 161.71732955])), (21213, array([157.09577465, 135.03521127])), (27571, array([198.75      , 138.72477064])), (30109, array([148.31234867, 203.01210654])), (30451, array([175.01140684, 173.99746515])), (41144, array([222.85018727, 184.62921348])), (42792, array([189.97546012, 225.25357873])), (60716, array([245.31620553, 247.50197628]))]
115,114,  127,161,  157,135,  198,138,  148,203,  175,173,  222,184,  189,225,  245,247


In [11]:
print(len(sizes))

4793


### cut cells from yolo images/txts

In [ ]:
import os
from PIL import Image


def scan_files(directory, prefix=None, postfix=None):
    files_list = []
    for root, sub_dirs, files in os.walk(directory):
        for special_file in files:
            if postfix:
                if special_file.endswith(postfix):
                    files_list.append(os.path.join(root, special_file))
            elif prefix:
                if special_file.startswith(prefix):
                    files_list.append(os.path.join(root, special_file))
            else:
                files_list.append(os.path.join(root, special_file))
    return files_list

def cut_cell(img_name, txt_name, save_path):
    img = Image.open(img_name)
    w,h = img.size
    labels = []
    with open(txt_name, 'r') as f:
        for line in f.readlines():
            tokens = line.strip().split()
            cx, cy = float(tokens[1]), float(tokens[2])
            w_, h_ = float(tokens[3]), float(tokens[4])
            xmin, ymin = int((cx-w_/2)*w), int((cy-h_/2)*h)
            xmax, ymax = int((cx+w_/2)*w), int((cy+h_/2)*h)
            img.crop((xmin, ymin, xmax, ymax)).save(os.path.join(save_path, "{}_{}_{}_{}.jpg".format(xmin, ymin, xmax, ymax)))
    img.close()
    

    
image_dir = "/home/hdd0/Develop/tct/point_detect_excess/TRI-deep-l-608/train"
save_path = "/home/hdd0/Develop/tct/point_detect_excess/TRI-deep-l-608/cells"

img_names = scan_files(image_dir, postfix=".bmp")
os.makedirs(save_path, exist_ok=True)
for img_name in img_names:
    txt_name = os.path.splitext(img_name)[0] + ".txt"
    cut_cell(img_name, txt_name, save_path)

#### remove unwanted, small cells

In [ ]:
import os
import shutil

In [ ]:
image_dir = "/home/hdd0/Develop/tct/point_detect_excess/TRI-deep-l-608/train"

img_names = scan_files(image_dir, postfix=".bmp")
for img_name in img_names:
    txt_name = os.path.splitext(img_name)[0] + ".txt"
    xml_name = os.path.splitext(img_name)[0] + ".xml"
    to_delete = False
    with open(txt_name, 'r') as f:
        for line in f.readlines():
            tokens = line.strip().split()
            if float(tokens[3])*608 < 10 or float(tokens[4])*608 < 10:
                to_delete = True
    if to_delete:
        os.remove(img_name)
        os.remove(txt_name)
        os.remove(xml_name)